# MusicGen
Welcome to MusicGen's demo jupyter notebook. Here you will find a series of self-contained examples of how to use MusicGen in different settings.

First, we start by initializing MusicGen, you can choose a model from the following selection:
1. `facebook/musicgen-small` - 300M transformer decoder.
2. `facebook/musicgen-medium` - 1.5B transformer decoder.
3. `facebook/musicgen-melody` - 1.5B transformer decoder also supporting melody conditioning.
4. `facebook/musicgen-large` - 3.3B transformer decoder.

We will use the `facebook/musicgen-small` variant for the purpose of this demonstration.

In [1]:
from audiocraft.models import MusicGen
#from audiocraft.models import MultiBandDiffusion
import torch, torchaudio
from audiocraft.utils.notebook import display_audio
from audiocraft.data.audio import audio_write

USE_DIFFUSION_DECODER = False
# Using small model, better results would be obtained with `medium` or `large`.
model = MusicGen.get_pretrained('facebook/musicgen-small')
#if USE_DIFFUSION_DECODER:
#    mbd = MultiBandDiffusion.get_mbd_musicgen()

/home/nao/anaconda3/envs/mgen/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Next, let us configure the generation parameters. Specifically, you can control the following:
* `use_sampling` (bool, optional): use sampling if True, else do argmax decoding. Defaults to True.
* `top_k` (int, optional): top_k used for sampling. Defaults to 250.
* `top_p` (float, optional): top_p used for sampling, when set to 0 top_k is used. Defaults to 0.0.
* `temperature` (float, optional): softmax temperature parameter. Defaults to 1.0.
* `duration` (float, optional): duration of the generated waveform. Defaults to 30.0.
* `cfg_coef` (float, optional): coefficient used for classifier free guidance. Defaults to 3.0.

When left unchanged, MusicGen will revert to its default parameters.

In [2]:
output_duration = 12

model.set_generation_params(
    use_sampling=True,
    top_k=250,
    duration=output_duration,
    temperature=1.0
)

Next, we can go ahead and start generating music using one of the following modes:
* Unconditional samples using `model.generate_unconditional`
* Music continuation using `model.generate_continuation`
* Text-conditional samples using `model.generate`
* Melody-conditional samples using `model.generate_with_chroma`

### Music Continuation

In [5]:
from madmom.features.downbeats import DBNDownBeatTrackingProcessor
from madmom.features.downbeats import RNNDownBeatProcessor
import os
import soundfile as sf
import pyrubberband as pyrb
import librosa

proc = DBNDownBeatTrackingProcessor(beats_per_bar=4, fps = 100, verbose=False)

def extract_loop(file_path, desired_bpm, num_bars=2):
    try:
        _, sr = librosa.core.load(file_path, sr=None) # sr = None will retrieve the original sampling rate = 44100
    except:
        print('load file failed')
        return None
    try:
        act = RNNDownBeatProcessor(verbose=False)(file_path)
        down_beat=proc(act, verbose=False) # [..., 2] 2d-shape numpy array
    except Exception as exp:
        print('except happended', exp)
        return None
    count = 0
    name = file_path.replace('.wav', '')
    for i in range(down_beat.shape[0]):
        if down_beat[i][1] == 1 and i + 4*num_bars < down_beat.shape[0] and down_beat[i+4*num_bars][1] == 1:
            start_time = down_beat[i][0]
            end_time = down_beat[i + 4*num_bars][0]
            count += 1
            out_path = os.path.join("./", f'{name}_{count}.wav')
            y_one_bar, _ = librosa.core.load(file_path, offset=start_time, duration = end_time - start_time, sr=None)
            desired_duration = 60./desired_bpm * (4*num_bars)
            y_stretch = pyrb.time_stretch(y_one_bar, sr,  (end_time - start_time) / desired_duration)
            sf.write(out_path, y_stretch, sr)
            return out_path

In [6]:
# You can also use any audio from a file. Make sure to trim the file if it is too long!
from audiocraft.data.audio import audio_write

prompt_waveform, prompt_sr = torchaudio.load("../assets/beat.wav")
prompt_duration = 4
prompt_waveform = prompt_waveform[..., :int(prompt_duration * prompt_sr)]
output = model.generate_continuation(prompt_waveform, prompt_sample_rate=prompt_sr, descriptions=["beat with piano"], progress=True, return_tokens=True)
print(output[0].shape)
sr = model.sample_rate
start_pos = prompt_duration * sr 
display_audio(output[0][:,:,start_pos:], sample_rate=32000)

file_path = "output"
output_data = output[0].cpu().squeeze()[start_pos:]
audio_write(file_path, output_data, sr, strategy="loudness", loudness_compressor=True)
#print(output_data.shape)
#sf.write(file_path, output_data, sr)
extract_loop(file_path + ".wav", 160)

torch.Size([1, 1, 384000])


/home/nao/anaconda3/envs/mgen/lib/python3.8/site-packages/madmom/features/downbeats.py:287: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  best = np.argmax(np.asarray(results)[:, 1])


'./output_1.wav'

In [19]:
def _loop_gen(filepath, duration, bpm, description="jazzy beat"):
    prompt_waveform, prompt_sr = torchaudio.load(filepath)
    prompt_duration = duration/1000.
    prompt_waveform = prompt_waveform[..., :int(prompt_duration * prompt_sr)]
    output = model.generate_continuation(prompt_waveform, prompt_sample_rate=prompt_sr, descriptions=[description], progress=True, return_tokens=True)
    print(output[0].shape)
    sr = model.sample_rate
    start_pos = int(prompt_duration * sr) 
    display_audio(output[0][:,:,start_pos:], sample_rate=32000)

    file_path = "/mnt/c/tmp/output"
    output_data = output[0].cpu().squeeze()[start_pos:]
    audio_write(file_path, output_data, sr, strategy="loudness", loudness_compressor=True)
    outpath = extract_loop(file_path + ".wav", bpm)
    return outpath

In [20]:

# %%
import os 
import time
from threading import Thread
from pythonosc import dispatcher
from pythonosc import osc_server, udp_client

def loop_gen(unused_addr, filepath, duration, bpm):
    print(filepath)
    if os.path.exists(filepath):
        outpath = _loop_gen(filepath, duration, bpm)
        if outpath is not None:    
            client.send_message("/generated", (outpath))
    else:
        print("file not found", filepath)

dispatcher = dispatcher.Dispatcher()
dispatcher.map("/loop_gen", loop_gen)

#%%

#client = udp_client.SimpleUDPClient('127.0.0.1', 10018)
client = udp_client.SimpleUDPClient('192.168.3.17', 10018)


def beacon():
    while True:
        client.send_message("/heartbeat", 1)
        time.sleep(1.0)
t1 = Thread(target = beacon)
t1.setDaemon(True)
t1.start()

server = osc_server.ThreadingOSCUDPServer(
    ('172.17.140.208', 10023), dispatcher)
print("Serving on {}".format(server.server_address))
server.serve_forever()


Serving on ('172.17.140.208', 10023)


### Text-conditional Generation

In [ ]:
from audiocraft.utils.notebook import display_audio

output = model.generate(
    descriptions=[
        #'80s pop track with bassy drums and synth',
        #'90s rock song with loud guitars and heavy drums',
        #'Progressive rock drum and bass solo',
        #'Punk Rock song with loud drum and power guitar',
        #'Bluesy guitar instrumental with soulful licks and a driving rhythm section',
        #'Jazz Funk song with slap bass and powerful saxophone',
        'drum and bass beat with intense percussions'
    ],
    progress=True, return_tokens=True
)
display_audio(output[0], sample_rate=32000)
if USE_DIFFUSION_DECODER:
    out_diffusion = mbd.tokens_to_wav(output[1])
    display_audio(out_diffusion, sample_rate=32000)

### Melody-conditional Generation

In [ ]:
import torchaudio
from audiocraft.utils.notebook import display_audio

model = MusicGen.get_pretrained('facebook/musicgen-melody')
model.set_generation_params(duration=8)

melody_waveform, sr = torchaudio.load("../assets/bach.mp3")
melody_waveform = melody_waveform.unsqueeze(0).repeat(2, 1, 1)
output = model.generate_with_chroma(
    descriptions=[
        '80s pop track with bassy drums and synth',
        '90s rock song with loud guitars and heavy drums',
    ],
    melody_wavs=melody_waveform,
    melody_sample_rate=sr,
    progress=True, return_tokens=True
)
display_audio(output[0], sample_rate=32000)
if USE_DIFFUSION_DECODER:
    out_diffusion = mbd.tokens_to_wav(output[1])
    display_audio(out_diffusion, sample_rate=32000)